In [1]:
# Importing packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
# Initializing parameters

jobs = 4 # Number of jobs
machines = 6 # Number of machines
processing_time = [6,7,17,9,20,13]
v = 2 # Number of agvs
travel_time = [[0,3,4,5,6,7],[3,0,1,2,3,4],[4,1,0,1,2,3],[5,2,1,0,1,2],[6,3,2,1,0,1],[7,4,3,2,1,0]]
operations = [(i, j) for i in range(1,jobs+1) for j in range(1, machines + 1)]  # Example operations
l = 100000

In [3]:
# Initializing a model
model = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-01


In [4]:
# Decision variables
C_max = model.addVar( vtype = GRB.CONTINUOUS, name = 'C_max')
start_time_operation = model.addVars(range(1, jobs+1) , range( 1 , machines+1) , vtype = GRB.CONTINUOUS, lb=0, name = 'start_time_operation')
start_time_transportation = model.addVars(range(1 , jobs+1) , range( 1 , machines+1) , vtype = GRB.CONTINUOUS, lb=0, name = 'start_time_transportation') #Transportation for Job i and operatation j
y = model.addVars(operations, operations, vtype=GRB.BINARY, name="For operation precedence")
z = model.addVars(operations, operations, vtype=GRB.BINARY, name="For transportation precedence")

for i in range(1,jobs+1) :
    for j in range(1, machines+1) :
        for r in range(1, v+1) :
            x = model.addVars(range(1, jobs+1) , range( 1 , machines+1), range(1, v+1), vtype=GRB.BINARY, name="transportation by agv for job i and operation j binary variable")

In [5]:
#Adding constraints

#Constraint 1.1
for (i,j) in operations:
    if j<machines :
        model.addConstr(start_time_operation[i,j+1] >= start_time_operation[i,j] + processing_time[j] , name = f"ensuring sequence from {i}_{j}")

#Constraint 1.2 & 1.3
for (i,j) in operations:
    for (i_prime,j_prime) in operations:
        if (i,j) != (i_prime,j_prime):
            model.addConstr(start_time_operation[i,j] + l * y[i,j,i_prime,j_prime] >= start_time_operation[i_prime,j_prime] + processing_time[j_prime-1],name = f"seq{i_prime, j_prime} to seq{i, j}" )
            model.addConstr(start_time_operation[i_prime,j_prime] + l * (1 - y[i,j,i_prime,j_prime]) >= start_time_operation[i,j] + processing_time[j-1],name = f"seq{i, j} to seq{i_prime, j_prime}" ) 

#Constraint 1.4 & 1.5
for (i,j) in operations:
    if (j >=1) & (j < machines):
        model.addConstr(start_time_operation[i,j] + processing_time[j-1] <= start_time_transportation[i,j+1])
    if (j >=2) & (j < machines+1):
        model.addConstr(start_time_transportation[i,j] + travel_time[j-2][j-1] <= start_time_operation[i,j])

#Constraint 1.6
for (i,j) in operations:
    for (i_prime,j_prime) in operations:
        if (i,j) != (i_prime, j_prime):
            if (j >=2) & (j < machines+1) & (j_prime >=2) & (j_prime < machines+1):
                model.addConstr(start_time_transportation[i_prime,j_prime] + l* (1- z[i,j,i_prime,j_prime]) >= start_time_transportation[i,j] + travel_time[j-2][j-1] + travel_time[j-1][j_prime-2])

#Constraint 1.7 is divided as two to follow linearity (1.8 & 1.9)
for (i,j) in operations:
    for (i_prime,j_prime) in operations:
        for r in range(1, v+1):
            if (i,j) != (i_prime, j_prime):
                model.addConstr(z[i,j,i_prime,j_prime] + z[i_prime,j_prime, i, j] >= x[i,j,r] + x[i_prime,j_prime,r] - 1)
                model.addConstr(z[i,j,i_prime,j_prime] + z[i_prime,j_prime, i, j] <= x[i,j,r] - x[i_prime,j_prime,r] + 1)

#Constraint 1.10
for (i,j) in operations:
    model.addConstr(gp.quicksum(x[i, j, r] for r in range(1,v+1)) == 1, name=f"task_assign_exactly_one_agv_{i}_{j}_{r}")

#Constraint 1.11
for (i,j) in operations:
    model.addConstr(start_time_operation[i,j] + processing_time[j-1] <= C_max)

In [6]:
# Objective

model.setObjective(C_max, GRB.MINIMIZE)

# time limit of 3 minutes

model.setParam(GRB.Param.TimeLimit, 180)

# Optimization
model.optimize()

Set parameter TimeLimit to value 180
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3800 rows, 3505 columns and 13500 nonzeros
Model fingerprint: 0xfcf55803
Variable types: 49 continuous, 3456 integer (3456 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 1164 rows and 2442 columns
Presolve time: 0.15s
Presolved: 2636 rows, 1063 columns, 8572 nonzeros
Variable types: 45 continuous, 1018 integer (1018 binary)

Root relaxation: objective 9.800000e+01, 240 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/No

In [7]:
if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
    print(f"Minimized Makespan: {model.objVal}")

    # For getting all decision variables
    for decisionvariable in model.getVars():
        print(f"{decisionvariable.VarName} = {decisionvariable.X}")

Minimized Makespan: 287.99999948272784
C_max = 287.99999948272784
start_time_operation[1,1] = 17.999999949941405
start_time_operation[1,2] = 30.999999949941415
start_time_operation[1,3] = 102.99999994994143
start_time_operation[1,4] = 137.99999994994147
start_time_operation[1,5] = 215.99999997139352
start_time_operation[1,6] = 274.99999948272784
start_time_operation[2,1] = -4.672526393403283e-08
start_time_operation[2,2] = 37.999999949941426
start_time_operation[2,3] = 85.99999994994143
start_time_operation[2,4] = 128.9999999499414
start_time_operation[2,5] = 175.99999997139355
start_time_operation[2,6] = 261.99999948272784
start_time_operation[3,1] = 5.999999949941403
start_time_operation[3,2] = 44.99999994994142
start_time_operation[3,3] = 68.99999994994143
start_time_operation[3,4] = 146.99999994994144
start_time_operation[3,5] = 195.99999997139352
start_time_operation[3,6] = 235.99999997139352
start_time_operation[4,1] = 11.999999949941405
start_time_operation[4,2] = 23.99999994994